In [ ]:
import os
import logging
import asyncio
from datetime import datetime, timedelta, timezone
from collections import defaultdict
import threading
from dotenv import load_dotenv

from telegram import (
    Update,
    InlineKeyboardButton,
    InlineKeyboardMarkup,
    InputMediaPhoto,
    InputMediaVideo,
    InputMediaDocument,
    constants,
    BotCommand,
)
from telegram.ext import (
    Application,
    CommandHandler,
    MessageHandler,
    CallbackQueryHandler,
    ContextTypes,
    filters,
    ApplicationBuilder,
    JobQueue
)
from telegram.error import TelegramError, BadRequest

from pymongo import MongoClient
from pymongo.errors import CollectionInvalid, PyMongoError

# Flask for health check
from flask import Flask, jsonify

# Load environment variables from .env file
load_dotenv()

# --- Configuration and Environment Variables ---
BOT_TOKEN = os.getenv("BOT_TOKEN")
MONGO_URI = os.getenv("MONGO_URI")
APPROVAL_CHANNEL_ID = int(os.getenv("APPROVAL_CHANNEL_ID"))
# ADMIN_IDS should be a comma-separated string, convert to list of ints
INITIAL_ADMIN_IDS = [int(uid) for uid in os.getenv("ADMIN_IDS", "").split(',') if uid.strip()]

# Set up logging
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)
logger = logging.getLogger(__name__)

# --- MongoDB Setup ---
try:
    client = MongoClient(MONGO_URI)
    db = client.get_database()  # Gets the database specified in the MONGO_URI
    users_collection = db["users"]
    relayed_messages_collection = db["relayed_messages"]
    settings_collection = db["settings"] # For admin list, recurring messages
    logger.info("Connected to MongoDB successfully.")

    # Create indexes for efficient querying
    users_collection.create_index([("_id", 1)], unique=True)
    users_collection.create_index([("is_approved", 1)])
    users_collection.create_index([("is_whitelisted", 1)])
    users_collection.create_index([("is_banned", 1)])
    users_collection.create_index([("last_active", 1)])
    users_collection.create_index([("total_sent_media", -1)])

    relayed_messages_collection.create_index([("original_message_id", 1)])
    relayed_messages_collection.create_index([("original_sender_id", 1)])
    relayed_messages_collection.create_index([("relayed_to.chat_id", 1), ("relayed_to.message_id", 1)])
    relayed_messages_collection.create_index([("original_sender_chat_id", 1), ("original_message_id", 1)], unique=True)

except PyMongoError as e:
    logger.error(f"Failed to connect to MongoDB: {e}")
    exit(1) # Exit if cannot connect to DB

# --- Global Variables & Constants ---
MEDIA_GROUP_PROCESSING_DELAY = 1.5 # Seconds to wait for media group parts to arrive
MEDIA_GROUP_CACHE = defaultdict(list)
LAST_MESSAGE_TIMESTAMP = defaultdict(datetime.now) # To track media group arrival times
BATCH_SIZE = 10 # Number of users to send all media to in one go

# Inactivity thresholds
INACTIVITY_PERIOD_DAYS = 7
REQUIRED_WEEKLY_MEDIA = 25

# --- Utility Functions ---

def is_admin(user_id: int) -> bool:
    """Checks if a user is an admin."""
    admin_settings = settings_collection.find_one({"_id": "admin_ids"})
    if admin_settings:
        return user_id in admin_settings.get("ids", [])
    return user_id in INITIAL_ADMIN_IDS # Fallback to initial env var if DB is empty

async def get_user_data(user_id: int) -> dict:
    """Fetches user data from MongoDB."""
    return users_collection.find_one({"_id": user_id})

async def add_or_update_user(user_id: int, user_data: dict, is_new_user: bool = False):
    """Adds a new user or updates an existing one."""
    current_time = datetime.now(timezone.utc)
    update_fields = {
        "username": user_data.get("username"),
        "first_name": user_data.get("first_name"),
        "last_name": user_data.get("last_name"),
        "last_active": current_time,
    }

    if is_new_user:
        insert_data = {
            "_id": user_id,
            "is_approved": False,
            "is_whitelisted": False,
            "is_banned": False,
            "weekly_media_count": 0,
            "total_sent_media": 0,
            "is_admin": False, # Initially false, managed by /promote
            **update_fields
        }
        try:
            users_collection.insert_one(insert_data)
            logger.info(f"New user added: {user_id} - {user_data.get('username')}")
        except PyMongoError as e:
            logger.error(f"Error adding new user {user_id}: {e}")
    else:
        try:
            users_collection.update_one(
                {"_id": user_id},
                {"$set": update_fields}
            )
        except PyMongoError as e:
            logger.error(f"Error updating user {user_id}: {e}")

async def update_user_activity(user_id: int):
    """Updates last_active timestamp for a user."""
    try:
        users_collection.update_one(
            {"_id": user_id},
            {"$set": {"last_active": datetime.now(timezone.utc)}}
        )
    except PyMongoError as e:
        logger.error(f"Error updating activity for user {user_id}: {e}")

async def update_user_media_count(user_id: int, count: int = 1):
    """Increments media count for a user."""
    try:
        users_collection.update_one(
            {"_id": user_id},
            {"$inc": {"weekly_media_count": count, "total_sent_media": count}}
        )
    except PyMongoError as e:
        logger.error(f"Error updating media count for user {user_id}: {e}")

async def set_user_status(user_id: int, status_type: str, value: bool):
    """Sets a boolean status (approved, whitelisted, banned, admin) for a user."""
    try:
        users_collection.update_one(
            {"_id": user_id},
            {"$set": {status_type: value}}
        )
    except PyMongoError as e:
        logger.error(f"Error setting status {status_type} for user {user_id}: {e}")

async def delete_user_from_db(user_id: int):
    """Deletes a user's data from the database."""
    try:
        users_collection.delete_one({"_id": user_id})
        # Also clean up any relay records where this user was a recipient
        relayed_messages_collection.update_many(
            {},
            {"$pull": {"relayed_to": {"chat_id": user_id}}}
        )
        # Optionally, delete messages from relayed_messages if sender_id matches
        relayed_messages_collection.delete_many({"original_sender_id": user_id})
        logger.info(f"User {user_id} and their associated relay data deleted from DB.")
    except PyMongoError as e:
        logger.error(f"Error deleting user {user_id} from DB: {e}")

async def get_active_users_for_relay() -> list[dict]:
    """Retrieves active (approved, not banned, not inactive) users for message relaying."""
    current_time = datetime.now(timezone.utc)
    # Define criteria for active users:
    # 1. is_approved is True
    # 2. is_banned is False
    # 3. (is_whitelisted is True) OR (last_active is within INACTIVITY_PERIOD_DAYS OR weekly_media_count >= REQUIRED_WEEKLY_MEDIA)
    active_users = users_collection.find({
        "is_approved": True,
        "is_banned": False,
        "$or": [
            {"is_whitelisted": True},
            {"last_active": {"$gte": current_time - timedelta(days=INACTIVITY_PERIOD_DAYS)}},
            {"weekly_media_count": {"$gte": REQUIRED_WEEKLY_MEDIA}}
        ]
    })
    return list(active_users)

async def store_relayed_message_mapping(
    original_chat_id: int, original_message_id: int, original_sender_id: int, recipient_id: int, relayed_message_id: int
):
    """Stores the mapping between original message and relayed messages."""
    doc_id = f"{original_chat_id}_{original_message_id}"
    try:
        relayed_messages_collection.update_one(
            {"_id": doc_id},
            {
                "$set": {
                    "original_message_id": original_message_id,
                    "original_sender_chat_id": original_chat_id,
                    "original_sender_id": original_sender_id,
                    "timestamp": datetime.now(timezone.utc)
                },
                "$addToSet": { # Use addToSet to avoid duplicate recipient entries
                    "relayed_to": {"chat_id": recipient_id, "message_id": relayed_message_id}
                }
            },
            upsert=True # Create new document if _id doesn't exist
        )
    except PyMongoError as e:
        logger.error(f"Error storing relayed message mapping for {doc_id}: {e}")

async def get_relayed_message_mapping(original_chat_id: int, original_message_id: int) -> dict:
    """Retrieves the mapping for a given original message."""
    doc_id = f"{original_chat_id}_{original_message_id}"
    return relayed_messages_collection.find_one({"_id": doc_id})

async def delete_relayed_messages_from_db_record(original_chat_id: int, original_message_id: int):
    """Deletes the record of relayed messages for a specific original message."""
    doc_id = f"{original_chat_id}_{original_message_id}"
    try:
        relayed_messages_collection.delete_one({"_id": doc_id})
        logger.info(f"Relayed message record for {doc_id} deleted from DB.")
    except PyMongoError as e:
        logger.error(f"Error deleting relayed message record {doc_id}: {e}")

async def get_all_admin_ids() -> list[int]:
    """Retrieves the list of admin IDs from DB or uses initial env var."""
    admin_doc = settings_collection.find_one({"_id": "admin_ids"})
    if admin_doc and "ids" in admin_doc:
        return admin_doc["ids"]
    else:
        # Initialize with ENV_ADMIN_IDS if not found in DB
        if INITIAL_ADMIN_IDS:
            settings_collection.update_one(
                {"_id": "admin_ids"},
                {"$set": {"ids": INITIAL_ADMIN_IDS}},
                upsert=True
            )
        return INITIAL_ADMIN_IDS

async def add_admin_to_db(user_id: int):
    """Adds a user ID to the admin list in the database."""
    current_admins = await get_all_admin_ids()
    if user_id not in current_admins:
        current_admins.append(user_id)
        settings_collection.update_one(
            {"_id": "admin_ids"},
            {"$set": {"ids": current_admins}},
            upsert=True
        )
        logger.info(f"User {user_id} promoted to admin.")

async def remove_admin_from_db(user_id: int):
    """Removes a user ID from the admin list in the database."""
    current_admins = await get_all_admin_ids()
    if user_id in current_admins:
        current_admins.remove(user_id)
        settings_collection.update_one(
            {"_id": "admin_ids"},
            {"$set": {"ids": current_admins}},
            upsert=True
        )
        logger.info(f"User {user_id} demoted from admin.")


# --- Command Handlers ---

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Handles the /start command."""
    user = update.effective_user
    user_data = await get_user_data(user.id)

    if not user_data:
        # New user
        await add_or_update_user(user.id, user.to_dict(), is_new_user=True)
        keyboard = [[InlineKeyboardButton("Request Approval", callback_data=f"request_approval_{user.id}")]]
        reply_markup = InlineKeyboardMarkup(keyboard)
        await update.message.reply_text(
            "Welcome to the Relay Bot! To start receiving messages, please request approval.",
            reply_markup=reply_markup
        )
        logger.info(f"New user {user.id} started the bot. Awaiting approval.")
    else:
        # Existing user
        await add_or_update_user(user.id, user.to_dict()) # Update username/first_name etc.
        if user_data["is_approved"] and not user_data["is_banned"]:
            await update.message.reply_text("Welcome back! You are already approved and can send/receive messages.")
        elif user_data["is_banned"]:
            await update.message.reply_text("You are currently banned from using this bot.")
        elif not user_data["is_approved"]:
            keyboard = [[InlineKeyboardButton("Request Approval", callback_data=f"request_approval_{user.id}")]]
            reply_markup = InlineKeyboardMarkup(keyboard)
            await update.message.reply_text(
                "Your account is not yet approved or was made inactive. Please request approval again.",
                reply_markup=reply_markup
            )

async def request_approval_callback(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Handles the 'Request Approval' inline button callback."""
    query = update.callback_query
    user_id_to_approve = int(query.data.split('_')[-1])
    user_requesting = await get_user_data(user_id_to_approve)

    if not user_requesting:
        await query.answer("User data not found for this request.", show_alert=True)
        return

    full_name = f"{user_requesting.get('first_name', '')} {user_requesting.get('last_name', '')}".strip()
    username = user_requesting.get('username')
    user_info = f"ID: `{user_id_to_approve}`\nName: {full_name}\nUsername: @{username if username else 'N/A'}"

    keyboard = [
        [
            InlineKeyboardButton("Approve", callback_data=f"approve_{user_id_to_approve}"),
            InlineKeyboardButton("Reject", callback_data=f"reject_{user_id_to_approve}")
        ]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)

    try:
        await context.bot.send_message(
            chat_id=APPROVAL_CHANNEL_ID,
            text=f"New user approval request:\n{user_info}",
            reply_markup=reply_markup,
            parse_mode=constants.ParseMode.MARKDOWN
        )
        await query.answer("Your approval request has been sent to the admins. Please wait.")
        logger.info(f"Approval request sent for user {user_id_to_approve}.")
    except TelegramError as e:
        logger.error(f"Error sending approval request to channel: {e}")
        await query.answer("Failed to send approval request. Please try again later.", show_alert=True)

async def approval_action_callback(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Handles 'Approve'/'Reject' inline button callbacks in the approval channel."""
    query = update.callback_query
    admin_id = query.from_user.id

    if not await is_admin(admin_id):
        await query.answer("You are not authorized to perform this action.", show_alert=True)
        return

    action, user_id_str = query.data.split('_')
    user_id_target = int(user_id_str)
    user_target_data = await get_user_data(user_id_target)

    if not user_target_data:
        await query.answer("User not found in database.", show_alert=True)
        return

    if action == "approve":
        await set_user_status(user_id_target, "is_approved", True)
        await set_user_status(user_id_target, "is_banned", False) # Ensure not banned
        await set_user_status(user_id_target, "is_admin", False) # Ensure not admin by default
        await set_user_status(user_id_target, "is_whitelisted", False) # Ensure not whitelisted by default
        try:
            await context.bot.send_message(
                chat_id=user_id_target,
                text="Congratulations! Your request has been approved. You can now send and receive messages."
            )
            await query.edit_message_text(f"Approved user {user_id_target} by admin {admin_id}.")
            logger.info(f"User {user_id_target} approved by admin {admin_id}.")
        except TelegramError as e:
            logger.error(f"Failed to notify approved user {user_id_target}: {e}")
            await query.edit_message_text(f"Approved user {user_id_target} by admin {admin_id}. (Failed to notify user)")
    elif action == "reject":
        await set_user_status(user_id_target, "is_approved", False)
        try:
            await context.bot.send_message(
                chat_id=user_id_target,
                text="Your approval request has been rejected. You cannot use the bot at this time."
            )
            await query.edit_message_text(f"Rejected user {user_id_target} by admin {admin_id}.")
            logger.info(f"User {user_id_target} rejected by admin {admin_id}.")
        except TelegramError as e:
            logger.error(f"Failed to notify rejected user {user_id_target}: {e}")
            await query.edit_message_text(f"Rejected user {user_id_target} by admin {admin_id}. (Failed to notify user)")

    await query.answer()

async def service_message_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Admin command: Sets a recurring message."""
    if not await is_admin(update.effective_user.id):
        await update.message.reply_text("You are not authorized to use this command.")
        return

    message_text = " ".join(context.args)
    if not message_text:
        await update.message.reply_text("Usage: `/service_message <message>`", parse_mode=constants.ParseMode.MARKDOWN)
        return

    settings_collection.update_one(
        {"_id": "recurring_message"},
        {"$set": {"message_text": message_text}},
        upsert=True
    )
    # Schedule the job if it's not already scheduled
    job_name = "send_recurring_message_job"
    for job in context.job_queue.get_jobs_by_name(job_name):
        job.schedule_removal()
    context.job_queue.run_repeating(send_recurring_message, interval=3 * 3600, first=0, name=job_name) # Every 3 hours

    await update.message.reply_text(f"Service message set and will be sent every 3 hours: \n`{message_text}`",
                                    parse_mode=constants.ParseMode.MARKDOWN)
    logger.info(f"Admin {update.effective_user.id} set recurring message.")

async def whitelist_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Admin command: Adds a user to the whitelist."""
    if not await is_admin(update.effective_user.id):
        await update.message.reply_text("You are not authorized to use this command.")
        return

    if not context.args or not context.args[0].isdigit():
        await update.message.reply_text("Usage: `/whitelist <user_id>`", parse_mode=constants.ParseMode.MARKDOWN)
        return

    user_id = int(context.args[0])
    user_data = await get_user_data(user_id)

    if not user_data:
        await update.message.reply_text(f"User `{user_id}` not found in the database.", parse_mode=constants.ParseMode.MARKDOWN)
        return

    await set_user_status(user_id, "is_whitelisted", True)
    await update.message.reply_text(f"User `{user_id}` has been whitelisted.", parse_mode=constants.ParseMode.MARKDOWN)
    logger.info(f"Admin {update.effective_user.id} whitelisted user {user_id}.")

async def unwhitelist_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Admin command: Removes a user from the whitelist."""
    if not await is_admin(update.effective_user.id):
        await update.message.reply_text("You are not authorized to use this command.")
        return

    if not context.args or not context.args[0].isdigit():
        await update.message.reply_text("Usage: `/unwhitelist <user_id>`", parse_mode=constants.ParseMode.MARKDOWN)
        return

    user_id = int(context.args[0])
    user_data = await get_user_data(user_id)

    if not user_data:
        await update.message.reply_text(f"User `{user_id}` not found in the database.", parse_mode=constants.ParseMode.MARKDOWN)
        return

    await set_user_status(user_id, "is_whitelisted", False)
    await update.message.reply_text(f"User `{user_id}` has been removed from the whitelist.", parse_mode=constants.ParseMode.MARKDOWN)
    logger.info(f"Admin {update.effective_user.id} unwhitelisted user {user_id}.")

async def ban_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Admin command: Bans a user."""
    if not await is_admin(update.effective_user.id):
        await update.message.reply_text("You are not authorized to use this command.")
        return

    if not context.args or not context.args[0].isdigit():
        await update.message.reply_text("Usage: `/ban <user_id>`", parse_mode=constants.ParseMode.MARKDOWN)
        return

    user_id = int(context.args[0])
    user_data = await get_user_data(user_id)

    if not user_data:
        await update.message.reply_text(f"User `{user_id}` not found in the database.", parse_mode=constants.ParseMode.MARKDOWN)
        return

    await set_user_status(user_id, "is_banned", True)
    await update.message.reply_text(f"User `{user_id}` has been banned.", parse_mode=constants.ParseMode.MARKDOWN)
    logger.info(f"Admin {update.effective_user.id} banned user {user_id}.")

async def unban_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Admin command: Unbans a user."""
    if not await is_admin(update.effective_user.id):
        await update.message.reply_text("You are not authorized to use this command.")
        return

    if not context.args or not context.args[0].isdigit():
        await update.message.reply_text("Usage: `/unban <user_id>`", parse_mode=constants.ParseMode.MARKDOWN)
        return

    user_id = int(context.args[0])
    user_data = await get_user_data(user_id)

    if not user_data:
        await update.message.reply_text(f"User `{user_id}` not found in the database.", parse_mode=constants.ParseMode.MARKDOWN)
        return

    await set_user_status(user_id, "is_banned", False)
    await update.message.reply_text(f"User `{user_id}` has been unbanned.", parse_mode=constants.ParseMode.MARKDOWN)
    logger.info(f"Admin {update.effective_user.id} unbanned user {user_id}.")

async def stats_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Admin command: Displays user statistics."""
    if not await is_admin(update.effective_user.id):
        await update.message.reply_text("You are not authorized to use this command.")
        return

    all_users = users_collection.find().sort("total_sent_media", -1)
    stats_text = "📊 **User Statistics (Top 10 by Media Sent)** 📊\n\n"
    count = 0
    for user_doc in all_users:
        if count >= 10:
            break
        user_id = user_doc["_id"]
        full_name = f"{user_doc.get('first_name', '')} {user_doc.get('last_name', '')}".strip()
        username = user_doc.get('username')
        total_media = user_doc.get('total_sent_media', 0)
        status = []
        if user_doc.get('is_approved'): status.append('Approved')
        if user_doc.get('is_whitelisted'): status.append('Whitelisted')
        if user_doc.get('is_banned'): status.append('Banned')
        if user_doc.get('is_admin'): status.append('Admin')

        status_str = ", ".join(status) if status else "Inactive/Pending"

        stats_text += (
            f"**User ID:** `{user_id}`\n"
            f"**Name:** {full_name} (@{username if username else 'N/A'})\n"
            f"**Total Media Sent:** {total_media}\n"
            f"**Status:** {status_str}\n\n"
        )
        count += 1

    if count == 0:
        stats_text = "No users found in the database."

    await update.message.reply_text(stats_text, parse_mode=constants.ParseMode.MARKDOWN)
    logger.info(f"Admin {update.effective_user.id} requested stats.")

async def delete_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Admin command: Deletes a specific relayed message from all receipts."""
    if not await is_admin(update.effective_user.id):
        await update.message.reply_text("You are not authorized to use this command.")
        return

    if not update.message.reply_to_message:
        await update.message.reply_text("Please reply to a message sent by the bot to delete it from all recipients.")
        return

    # The admin is replying to a message *sent by the bot*.
    # We need to find the original message that this bot message corresponds to.
    # This requires looking up the relayed_messages_collection using the bot's message ID
    # and the chat ID where the message was sent (which is the admin's chat ID).

    replied_message_id = update.message.reply_to_message.message_id
    admin_chat_id = update.effective_chat.id

    # Find the record in relayed_messages_collection that contains this (admin_chat_id, replied_message_id) pair
    # Use find_one to get the first matching document
    relay_record = relayed_messages_collection.find_one({
        "relayed_to": {"$elemMatch": {"chat_id": admin_chat_id, "message_id": replied_message_id}}
    })

    if not relay_record:
        await update.message.reply_text("Could not find the original message record for this relayed message. It might be too old or not a relayed message.")
        return

    original_chat_id = relay_record["original_sender_chat_id"]
    original_message_id = relay_record["original_message_id"]
    recipients_to_delete = relay_record["relayed_to"]

    successful_deletions = 0
    failed_deletions = 0

    for recipient_info in recipients_to_delete:
        recipient_chat_id = recipient_info["chat_id"]
        message_id_to_delete = recipient_info["message_id"]
        try:
            await context.bot.delete_message(chat_id=recipient_chat_id, message_id=message_id_to_delete)
            successful_deletions += 1
        except TelegramError as e:
            logger.warning(f"Failed to delete message {message_id_to_delete} in chat {recipient_chat_id}: {e}")
            # If the chat_id where the message was originally relayed is the admin's own chat,
            # we should delete it from there as well, but only once.
            if recipient_chat_id != admin_chat_id or (recipient_chat_id == admin_chat_id and message_id_to_delete != replied_message_id):
                 failed_deletions += 1
            pass # Continue to next recipient even if one fails

    # Also delete the message from the admin's chat that was replied to
    try:
        await context.bot.delete_message(chat_id=admin_chat_id, message_id=replied_message_id)
        successful_deletions += 1 # Count admin's own message delete
    except TelegramError as e:
        logger.warning(f"Failed to delete replied message {replied_message_id} in admin chat {admin_chat_id}: {e}")
        failed_deletions += 1

    # Remove the record from the database after attempting deletions
    await delete_relayed_messages_from_db_record(original_chat_id, original_message_id)

    await update.message.reply_text(
        f"Attempted to delete the message.\n"
        f"Successful deletions: {successful_deletions}\n"
        f"Failed deletions: {failed_deletions}"
    )
    logger.info(f"Admin {admin_chat_id} attempted to delete message with original ID {original_message_id}. Success: {successful_deletions}, Fail: {failed_deletions}")


async def admin_message_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Admin command: Captures a message and sends it to the approval channel."""
    if not await is_admin(update.effective_user.id):
        await update.message.reply_text("You are not authorized to use this command.")
        return

    message_to_send = " ".join(context.args)
    if not message_to_send:
        await update.message.reply_text("Usage: `/admin <message>`", parse_mode=constants.ParseMode.MARKDOWN)
        return

    try:
        await context.bot.send_message(
            chat_id=APPROVAL_CHANNEL_ID,
            text=f"**Message from Admin {update.effective_user.first_name} (@{update.effective_user.username}):**\n\n{message_to_send}",
            parse_mode=constants.ParseMode.MARKDOWN
        )
        await update.message.reply_text("Message sent to approval channel.")
        logger.info(f"Admin {update.effective_user.id} sent message to approval channel.")
    except TelegramError as e:
        logger.error(f"Error sending admin message to approval channel: {e}")
        await update.message.reply_text("Failed to send message to approval channel.")

async def pin_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Admin command: Pins a specific message in each individual active user's chat."""
    if not await is_admin(update.effective_user.id):
        await update.message.reply_text("You are not authorized to use this command.")
        return

    if not update.message.reply_to_message:
        await update.message.reply_text("Please reply to the message you want to pin in all active users' chats.")
        return

    message_to_pin = update.message.reply_to_message
    active_users = await get_active_users_for_relay()
    successful_pins = 0
    failed_pins = 0

    for user_doc in active_users:
        user_id = user_doc["_id"]
        try:
            # Forward and then pin the forwarded message
            forwarded_message = await message_to_pin.forward(chat_id=user_id)
            await context.bot.pin_chat_message(chat_id=user_id, message_id=forwarded_message.message_id)
            successful_pins += 1
            await asyncio.sleep(0.1) # Small delay to avoid rate limits
        except TelegramError as e:
            logger.warning(f"Failed to pin message for user {user_id}: {e}")
            # If bot blocked by user, remove them from DB
            if "bot was blocked by the user" in str(e).lower() or "user not found" in str(e).lower():
                await delete_user_from_db(user_id)
            failed_pins += 1

    await update.message.reply_text(f"Attempted to pin the message in active users' chats.\n"
                                    f"Successful pins: {successful_pins}\n"
                                    f"Failed pins: {failed_pins}")
    logger.info(f"Admin {update.effective_user.id} attempted to pin a message. Success: {successful_pins}, Fail: {failed_pins}")


async def userinfo_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Admin-only command: Shows detailed info about a user."""
    if not await is_admin(update.effective_user.id):
        await update.message.reply_text("You are not authorized to use this command.")
        return

    if not context.args or not context.args[0].isdigit():
        await update.message.reply_text("Usage: `/userinfo <user_id>`", parse_mode=constants.ParseMode.MARKDOWN)
        return

    user_id = int(context.args[0])
    user_data = await get_user_data(user_id)

    if not user_data:
        await update.message.reply_text(f"User `{user_id}` not found in the database.", parse_mode=constants.ParseMode.MARKDOWN)
        return

    full_name = f"{user_data.get('first_name', '')} {user_data.get('last_name', '')}".strip()
    username = user_data.get('username')
    last_active = user_data.get('last_active')
    status_list = []
    if user_data.get('is_approved'): status_list.append("Approved")
    if user_data.get('is_whitelisted'): status_list.append("Whitelisted")
    if user_data.get('is_banned'): status_list.append("Banned")
    if user_data.get('is_admin'): status_list.append("Admin")
    if not user_data.get('is_approved') and not user_data.get('is_banned'): status_list.append("Pending/Inactive")

    status_info = ", ".join(status_list) if status_list else "Unknown"

    info_text = (
        f"**User Info for ID:** `{user_id}`\n"
        f"**Name:** {full_name} (@{username if username else 'N/A'})\n"
        f"**Total Messages:** {user_data.get('total_sent_media', 0)}\n"
        f"**Last Active:** {last_active.strftime('%Y-%m-%d %H:%M:%S UTC') if last_active else 'N/A'}\n"
        f"**Status:** {status_info}\n"
        f"**Weekly Media Count:** {user_data.get('weekly_media_count', 0)}"
    )

    await update.message.reply_text(info_text, parse_mode=constants.ParseMode.MARKDOWN)
    logger.info(f"Admin {update.effective_user.id} requested user info for {user_id}.")

async def promote_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Admin command: Promotes a user to admin status."""
    if not await is_admin(update.effective_user.id):
        await update.message.reply_text("You are not authorized to use this command.")
        return

    if not context.args or not context.args[0].isdigit():
        await update.message.reply_text("Usage: `/promote <user_id>`", parse_mode=constants.ParseMode.MARKDOWN)
        return

    user_id_to_promote = int(context.args[0])
    if await is_admin(user_id_to_promote):
        await update.message.reply_text(f"User `{user_id_to_promote}` is already an admin.", parse_mode=constants.ParseMode.MARKDOWN)
        return

    await add_admin_to_db(user_id_to_promote)
    await update.message.reply_text(f"User `{user_id_to_promote}` has been promoted to admin.", parse_mode=constants.ParseMode.MARKDOWN)
    logger.info(f"Admin {update.effective_user.id} promoted user {user_id_to_promote}.")

async def demote_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Admin command: Demotes a user from admin status."""
    if not await is_admin(update.effective_user.id):
        await update.message.reply_text("You are not authorized to use this command.")
        return

    if not context.args or not context.args[0].isdigit():
        await update.message.reply_text("Usage: `/demote <user_id>`", parse_mode=constants.ParseMode.MARKDOWN)
        return

    user_id_to_demote = int(context.args[0])
    if not await is_admin(user_id_to_demote):
        await update.message.reply_text(f"User `{user_id_to_demote}` is not an admin.", parse_mode=constants.ParseMode.MARKDOWN)
        return

    await remove_admin_from_db(user_id_to_demote)
    await update.message.reply_text(f"User `{user_id_to_demote}` has been demoted from admin.", parse_mode=constants.ParseMode.MARKDOWN)
    logger.info(f"Admin {update.effective_user.id} demoted user {user_id_to_demote}.")


# --- Message Relaying Logic ---

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Main message handler for relaying messages."""
    message = update.effective_message
    sender_user = update.effective_user
    sender_chat = update.effective_chat

    if not sender_user or sender_user.id == context.bot.id:
        return # Ignore messages from the bot itself or without a sender

    # Always update user's last active and basic info
    await add_or_update_user(sender_user.id, sender_user.to_dict())

    user_data = await get_user_data(sender_user.id)
    if not user_data or not user_data.get("is_approved") or user_data.get("is_banned"):
        # If not approved or banned, silently ignore or inform
        if not user_data:
            logger.info(f"Ignoring message from unknown user {sender_user.id}.")
            await update.message.reply_text("Please use `/start` to begin using the bot and request approval.")
        elif not user_data.get("is_approved"):
            logger.info(f"Ignoring message from unapproved user {sender_user.id}.")
            await update.message.reply_text("You are not yet approved. Please wait for admin approval or re-request if you were inactive.")
        elif user_data.get("is_banned"):
            logger.info(f"Ignoring message from banned user {sender_user.id}.")
            await update.message.reply_text("You are currently banned from using this bot.")
        return

    # Increment media count if it's a media message
    is_media = False
    if message.photo or message.video or message.document or message.animation or message.sticker or message.audio or message.voice:
        is_media = True
        await update_user_media_count(sender_user.id)

    # Update last active for any message
    await update_user_activity(sender_user.id)

    # --- Handle Media Groups ---
    if message.media_group_id:
        MEDIA_GROUP_CACHE[message.media_group_id].append(message)
        LAST_MESSAGE_TIMESTAMP[message.media_group_id] = datetime.now()
        # Schedule a job to process the media group after a short delay
        context.job_queue.run_once(
            process_media_group,
            MEDIA_GROUP_PROCESSING_DELAY,
            data={"media_group_id": message.media_group_id, "sender_id": sender_user.id},
            name=f"media_group_processor_{message.media_group_id}"
        )
        return # Wait for all parts of the media group

    # --- Handle Individual Messages (not part of a media group) ---
    await relay_individual_message(message, context, sender_user, sender_chat.id)


async def process_media_group(context: ContextTypes.DEFAULT_TYPE) -> None:
    """Processes a collected media group and relays it."""
    job_data = context.job.data
    media_group_id = job_data["media_group_id"]
    sender_user_id = job_data["sender_id"]

    # Ensure no other parts arrived during the delay
    if datetime.now() - LAST_MESSAGE_TIMESTAMP[media_group_id] < timedelta(seconds=MEDIA_GROUP_PROCESSING_DELAY - 0.1):
        # Another part likely arrived right before processing, reschedule
        context.job_queue.run_once(
            process_media_group,
            MEDIA_GROUP_PROCESSING_DELAY,
            data={"media_group_id": media_group_id, "sender_id": sender_user_id},
            name=f"media_group_processor_{media_group_id}"
        )
        return

    messages_in_group = MEDIA_GROUP_CACHE.pop(media_group_id, [])
    if not messages_in_group:
        return

    # Sort messages by message_id to maintain order if possible
    messages_in_group.sort(key=lambda m: m.message_id)

    # Prepare media for relaying
    media_to_send = []
    # Identify the original message to extract reply_to_message_id if any
    original_message = messages_in_group[0]
    # Keep track of individual items if they cannot be grouped by Telegram
    individual_media_to_send = []

    for msg in messages_in_group:
        if msg.photo:
            media_to_send.append(InputMediaPhoto(msg.photo[-1].file_id)) # Use last (largest) photo
        elif msg.video:
            media_to_send.append(InputMediaVideo(msg.video.file_id))
        elif msg.document:
            media_to_send.append(InputMediaDocument(msg.document.file_id))
        elif msg.animation:
            individual_media_to_send.append(("animation", msg.animation.file_id))
        elif msg.sticker:
            individual_media_to_send.append(("sticker", msg.sticker.file_id))
        elif msg.audio:
            individual_media_to_send.append(("audio", msg.audio.file_id))
        elif msg.voice:
            individual_media_to_send.append(("voice", msg.voice.file_id))
        elif msg.text: # Should not happen in a typical media group, but for robustness
             individual_media_to_send.append(("text", msg.text))

    # Get active users
    active_users = await get_active_users_for_relay()

    # Split active users into batches
    user_batches = [active_users[i:i + BATCH_SIZE] for i in range(0, len(active_users), BATCH_SIZE)]

    for batch in user_batches:
        for user_doc in batch:
            recipient_id = user_doc["_id"]
            if recipient_id == sender_user_id:
                continue # Don't relay back to sender

            try:
                # Handle replies for the *first message* in the group, as Telegram links replies to a single message.
                reply_to_relayed_message_id = None
                if original_message.reply_to_message:
                    # Look up the relayed message ID for the replied-to message in the recipient's chat
                    original_replied_to_chat_id = original_message.reply_to_message.chat.id # This is the chat ID where the *original* message was
                    original_replied_to_message_id = original_message.reply_to_message.message_id
                    relayed_mapping = await get_relayed_message_mapping(original_replied_to_chat_id, original_replied_to_message_id)

                    if relayed_mapping:
                        for entry in relayed_mapping.get("relayed_to", []):
                            if entry["chat_id"] == recipient_id:
                                reply_to_relayed_message_id = entry["message_id"]
                                break

                # Send media group first
                if media_to_send:
                    # Telegram's send_media_group doesn't support reply_to_message_id directly for albums
                    # We send the album, and if it was a reply, we send an additional text message
                    # indicating the reply context or a separate "replied to" message.
                    # Or, better, just send the album and let the user figure out the reply.
                    # The user explicitly asked for 'reply relaying feature so each user can see on which message a user has replied.'
                    # For media groups, this is tricky. We'll simply forward the group without explicit reply linking
                    # in the album itself, but individual media will try to link.

                    sent_messages = await context.bot.send_media_group(
                        chat_id=recipient_id,
                        media=media_to_send
                    )
                    # Store mapping for the first message in the group (representative)
                    if sent_messages:
                        await store_relayed_message_mapping(
                            original_message.chat.id,
                            original_message.message_id,
                            sender_user_id,
                            recipient_id,
                            sent_messages[0].message_id # Store the ID of the first message in the group
                        )
                    logger.info(f"Relayed media group from {sender_user_id} to {recipient_id}.")

                # Send individual media items
                for media_type, file_id in individual_media_to_send:
                    if media_type == "animation":
                        sent_message = await context.bot.send_animation(
                            chat_id=recipient_id,
                            animation=file_id,
                            reply_to_message_id=reply_to_relayed_message_id # This should work for individual messages
                        )
                    elif media_type == "sticker":
                        sent_message = await context.bot.send_sticker(
                            chat_id=recipient_id,
                            sticker=file_id,
                            reply_to_message_id=reply_to_relayed_message_id
                        )
                    elif media_type == "audio":
                        sent_message = await context.bot.send_audio(
                            chat_id=recipient_id,
                            audio=file_id,
                            reply_to_message_id=reply_to_relayed_message_id
                        )
                    elif media_type == "voice":
                        sent_message = await context.bot.send_voice(
                            chat_id=recipient_id,
                            voice=file_id,
                            reply_to_message_id=reply_to_relayed_message_id
                        )
                    elif media_type == "text":
                         sent_message = await context.bot.send_message(
                            chat_id=recipient_id,
                            text=file_id,
                            reply_to_message_id=reply_to_relayed_message_id
                        )
                    if sent_message:
                        await store_relayed_message_mapping(
                            original_message.chat.id,
                            original_message.message_id,
                            sender_user_id,
                            recipient_id,
                            sent_message.message_id
                        )
                        logger.info(f"Relayed individual media/text from {sender_user_id} to {recipient_id}.")

            except TelegramError as e:
                logger.warning(f"Failed to relay media group from {sender_user_id} to {recipient_id}: {e}")
                if "bot was blocked by the user" in str(e).lower() or "user not found" in str(e).lower():
                    await delete_user_from_db(recipient_id)
            await asyncio.sleep(0.05) # Small delay for rate limiting

        await asyncio.sleep(0.5) # Delay between batches

    if media_group_id in LAST_MESSAGE_TIMESTAMP:
        del LAST_MESSAGE_TIMESTAMP[media_group_id]


async def relay_individual_message(message, context, sender_user, original_sender_chat_id):
    """Relays an individual message (text, or single media not in a group)."""
    caption = None
    if message.text:
        # For text messages, write sender's name in the caption
        full_name = f"{sender_user.first_name} {sender_user.last_name or ''}".strip()
        caption = f"From: {full_name}\n\n{message.text}"
    else:
        # For media, send anonymously, so no sender name in caption
        caption = message.caption

    active_users = await get_active_users_for_relay()

    user_batches = [active_users[i:i + BATCH_SIZE] for i in range(0, len(active_users), BATCH_SIZE)]

    for batch in user_batches:
        for user_doc in batch:
            recipient_id = user_doc["_id"]
            if recipient_id == sender_user.id:
                continue # Don't relay back to sender

            try:
                # Handle replies
                reply_to_relayed_message_id = None
                if message.reply_to_message:
                    # Look up the relayed message ID for the replied-to message in the recipient's chat
                    original_replied_to_message_id = message.reply_to_message.message_id
                    relayed_mapping = await get_relayed_message_mapping(original_sender_chat_id, original_replied_to_message_id)

                    if relayed_mapping:
                        for entry in relayed_mapping.get("relayed_to", []):
                            if entry["chat_id"] == recipient_id:
                                reply_to_relayed_message_id = entry["message_id"]
                                break

                sent_message = None
                if message.photo:
                    sent_message = await context.bot.send_photo(
                        chat_id=recipient_id,
                        photo=message.photo[-1].file_id,
                        caption=caption,
                        reply_to_message_id=reply_to_relayed_message_id
                    )
                elif message.video:
                    sent_message = await context.bot.send_video(
                        chat_id=recipient_id,
                        video=message.video.file_id,
                        caption=caption,
                        reply_to_message_id=reply_to_relayed_message_id
                    )
                elif message.document:
                    sent_message = await context.bot.send_document(
                        chat_id=recipient_id,
                        document=message.document.file_id,
                        caption=caption,
                        reply_to_message_id=reply_to_relayed_message_id
                    )
                elif message.animation:
                    sent_message = await context.bot.send_animation(
                        chat_id=recipient_id,
                        animation=message.animation.file_id,
                        caption=caption,
                        reply_to_message_id=reply_to_relayed_message_id
                    )
                elif message.sticker:
                    sent_message = await context.bot.send_sticker(
                        chat_id=recipient_id,
                        sticker=message.sticker.file_id,
                        reply_to_message_id=reply_to_relayed_message_id
                    )
                elif message.audio:
                    sent_message = await context.bot.send_audio(
                        chat_id=recipient_id,
                        audio=message.audio.file_id,
                        caption=caption,
                        reply_to_message_id=reply_to_relayed_message_id
                    )
                elif message.voice:
                    sent_message = await context.bot.send_voice(
                        chat_id=recipient_id,
                        voice=message.voice.file_id,
                        caption=caption,
                        reply_to_message_id=reply_to_relayed_message_id
                    )
                elif message.text:
                    sent_message = await context.bot.send_message(
                        chat_id=recipient_id,
                        text=caption,
                        reply_to_message_id=reply_to_relayed_message_id
                    )
                # Add more message types as needed (e.g., location, contact, poll)

                if sent_message:
                    await store_relayed_message_mapping(
                        original_sender_chat_id,
                        message.message_id,
                        sender_user.id,
                        recipient_id,
                        sent_message.message_id
                    )
                    logger.info(f"Relayed message {message.message_id} from {sender_user.id} to {recipient_id}.")

            except TelegramError as e:
                logger.warning(f"Failed to relay message from {sender_user.id} to {recipient_id}: {e}")
                # Remove user from DB if bot blocked
                if "bot was blocked by the user" in str(e).lower() or "user not found" in str(e).lower():
                    await delete_user_from_db(recipient_id)
            await asyncio.sleep(0.05) # Small delay for rate limits

        await asyncio.sleep(0.5) # Delay between batches


# --- Job Queue Callbacks ---

async def check_inactivity(context: ContextTypes.DEFAULT_TYPE) -> None:
    """Scheduled job: Checks for inactive users and marks them as unapproved."""
    logger.info("Running inactivity check job.")
    current_time = datetime.now(timezone.utc)
    inactive_threshold = current_time - timedelta(days=INACTIVITY_PERIOD_DAYS)

    users_to_check = users_collection.find({
        "is_approved": True,
        "is_whitelisted": False,
        "is_banned": False # Do not check banned users for inactivity
    })

    for user_doc in users_to_check:
        user_id = user_doc["_id"]
        last_active = user_doc.get("last_active")
        weekly_media = user_doc.get("weekly_media_count", 0)

        if (last_active and last_active < inactive_threshold) and (weekly_media < REQUIRED_WEEKLY_MEDIA):
            await set_user_status(user_id, "is_approved", False) # Mark as unapproved
            try:
                await context.bot.send_message(
                    chat_id=user_id,
                    text="You have been marked inactive due to insufficient activity. Please use `/start` and request approval again to resume receiving messages."
                )
                logger.info(f"User {user_id} marked inactive and notified.")
            except TelegramError as e:
                logger.warning(f"Failed to notify inactive user {user_id}: {e}")
                if "bot was blocked by the user" in str(e).lower() or "user not found" in str(e).lower():
                    await delete_user_from_db(user_id)
            except Exception as e:
                logger.error(f"Unexpected error when processing inactivity for user {user_id}: {e}")


async def reset_weekly_media_count(context: ContextTypes.DEFAULT_TYPE) -> None:
    """Scheduled job: Resets weekly media count for all users."""
    logger.info("Resetting weekly media count for all users.")
    try:
        users_collection.update_many(
            {},
            {"$set": {"weekly_media_count": 0}}
        )
        logger.info("Weekly media count reset completed.")
    except PyMongoError as e:
        logger.error(f"Error resetting weekly media count: {e}")

async def send_recurring_message(context: ContextTypes.DEFAULT_TYPE) -> None:
    """Scheduled job: Sends the recurring service message to all active users."""
    logger.info("Sending recurring service message.")
    message_doc = settings_collection.find_one({"_id": "recurring_message"})
    if not message_doc or not message_doc.get("message_text"):
        logger.info("No recurring message set.")
        return

    message_text = message_doc["message_text"]
    active_users = await get_active_users_for_relay()
    successful_sends = 0
    failed_sends = 0

    for user_doc in active_users:
        user_id = user_doc["_id"]
        try:
            await context.bot.send_message(chat_id=user_id, text=f"**Service Message:**\n\n{message_text}",
                                            parse_mode=constants.ParseMode.MARKDOWN)
            successful_sends += 1
            await asyncio.sleep(0.05) # Small delay for rate limiting
        except TelegramError as e:
            logger.warning(f"Failed to send recurring message to user {user_id}: {e}")
            if "bot was blocked by the user" in str(e).lower() or "user not found" in str(e).lower():
                await delete_user_from_db(user_id)
            failed_sends += 1
    logger.info(f"Recurring message sent. Success: {successful_sends}, Failed: {failed_sends}")


async def daily_summary(context: ContextTypes.DEFAULT_TYPE) -> None:
    """Scheduled job: Sends daily summary to approval channel."""
    logger.info("Generating daily summary.")
    total_relayed_messages_today = relayed_messages_collection.count_documents({
        "timestamp": {"$gte": datetime.now(timezone.utc) - timedelta(days=1)}
    })

    top_active_users = users_collection.find(
        {"total_sent_media": {"$gt": 0}} # Only consider users who sent messages
    ).sort("total_sent_media", -1).limit(10)

    summary_text = f"📊 **Daily Summary - {datetime.now(timezone.utc).strftime('%Y-%m-%d')}** 📊\n\n"
    summary_text += f"Total relayed messages today: `{total_relayed_messages_today}`\n\n"
    summary_text += "**Top 10 Most Active Users (by Total Media Sent):**\n"

    user_count = 0
    for user_doc in top_active_users:
        user_id = user_doc["_id"]
        full_name = f"{user_doc.get('first_name', '')} {user_doc.get('last_name', '')}".strip()
        username = user_doc.get('username')
        total_media = user_doc.get('total_sent_media', 0)
        user_count += 1
        summary_text += f"{user_count}. `{user_id}` - {full_name} (@{username if username else 'N/A'}) - Sent: {total_media}\n"

    if user_count == 0:
        summary_text += "No active users to report today.\n"

    try:
        await context.bot.send_message(
            chat_id=APPROVAL_CHANNEL_ID,
            text=summary_text,
            parse_mode=constants.ParseMode.MARKDOWN
        )
        logger.info("Daily summary sent to approval channel.")
    except TelegramError as e:
        logger.error(f"Failed to send daily summary: {e}")

async def weekly_summary(context: ContextTypes.DEFAULT_TYPE) -> None:
    """Scheduled job: Sends weekly summary to approval channel."""
    logger.info("Generating weekly summary.")
    total_relayed_messages_week = relayed_messages_collection.count_documents({
        "timestamp": {"$gte": datetime.now(timezone.utc) - timedelta(weeks=1)}
    })

    top_active_users_week = users_collection.find(
        {"total_sent_media": {"$gt": 0}} # Only consider users who sent messages
    ).sort("total_sent_media", -1).limit(10)

    summary_text = f"📈 **Weekly Summary - {datetime.now(timezone.utc).strftime('%Y-%m-%d')}** 📈\n\n"
    summary_text += f"Total relayed messages this week: `{total_relayed_messages_week}`\n\n"
    summary_text += "**Top 10 Most Active Users (by Total Media Sent, Week)**\n"

    user_count = 0
    for user_doc in top_active_users_week:
        user_id = user_doc["_id"]
        full_name = f"{user_doc.get('first_name', '')} {user_doc.get('last_name', '')}".strip()
        username = user_doc.get('username')
        total_media = user_doc.get('total_sent_media', 0)
        user_count += 1
        summary_text += f"{user_count}. `{user_id}` - {full_name} (@{username if username else 'N/A'}) - Sent: {total_media}\n"

    if user_count == 0:
        summary_text += "No active users to report this week.\n"

    try:
        await context.bot.send_message(
            chat_id=APPROVAL_CHANNEL_ID,
            text=summary_text,
            parse_mode=constants.ParseMode.MARKDOWN
        )
        logger.info("Weekly summary sent to approval channel.")
    except TelegramError as e:
        logger.error(f"Failed to send weekly summary: {e}")

# --- Error Handling ---

async def error_handler(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Log Errors caused by Updates."""
    logger.error("Exception while handling an update:", exc_info=context.error)

    # Specific error handling for user blocking or account deletion
    if isinstance(context.error, TelegramError):
        # Check if the error message indicates bot was blocked by user or user not found
        error_message = str(context.error).lower()
        if "bot was blocked by the user" in error_message or "user not found" in error_message:
            user_id = None
            if update.effective_user:
                user_id = update.effective_user.id
            elif update.callback_query and update.callback_query.from_user:
                user_id = update.callback_query.from_user.id
            elif update.message and update.message.chat:
                user_id = update.message.chat.id

            if user_id:
                logger.warning(f"Bot blocked or user not found for {user_id}. Deleting user from DB.")
                await delete_user_from_db(user_id)
        # Handle other Telegram errors as needed

# --- Flask Health Check Server ---
app = Flask(__name__)

@app.route("/")
def health_check():
    """Basic health check endpoint."""
    return jsonify(status="ok", timestamp=datetime.now().isoformat())

def run_flask_app():
    """Runs the Flask app."""
    port = int(os.environ.get("PORT", 8000))
    app.run(host="0.0.0.0", port=port, debug=False)
    logger.info(f"Flask health check server started on port {port}")


# --- Main Function ---

def main() -> None:
    """Starts the bot."""
    # Ensure initial admins are in the database
    asyncio.run(get_all_admin_ids()) # This will ensure the admin_ids document exists and is populated

    application = Application.builder().token(BOT_TOKEN).build()
    job_queue = application.job_queue

    # Set bot commands for better UX
    application.bot.set_my_commands([
        BotCommand("start", "Start the bot and request approval"),
        BotCommand("service_message", "Admin: Set a recurring message"),
        BotCommand("whitelist", "Admin: Whitelist a user from inactivity cleaning"),
        BotCommand("unwhitelist", "Admin: Remove a user from whitelist"),
        BotCommand("ban", "Admin: Ban a user from receiving messages"),
        BotCommand("unban", "Admin: Unban a user"),
        BotCommand("stats", "Admin: Show user statistics"),
        BotCommand("delete", "Admin: Reply to a relayed message to delete it for all"),
        BotCommand("admin", "Admin: Send a message to the approval channel"),
        BotCommand("pin", "Admin: Reply to a message to pin it in all active chats"),
        BotCommand("userinfo", "Admin: Get detailed info about a user"),
        BotCommand("promote", "Admin: Promote a user to admin"),
        BotCommand("demote", "Admin: Demote a user from admin")
    ])

    # Command Handlers
    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("service_message", service_message_command))
    application.add_handler(CommandHandler("whitelist", whitelist_command))
    application.add_handler(CommandHandler("unwhitelist", unwhitelist_command))
    application.add_handler(CommandHandler("ban", ban_command))
    application.add_handler(CommandHandler("unban", unban_command))
    application.add_handler(CommandHandler("stats", stats_command))
    application.add_handler(CommandHandler("delete", delete_command))
    application.add_handler(CommandHandler("admin", admin_message_command))
    application.add_handler(CommandHandler("pin", pin_command))
    application.add_handler(CommandHandler("userinfo", userinfo_command))
    application.add_handler(CommandHandler("promote", promote_command))
    application.add_handler(CommandHandler("demote", demote_command))

    # Callback Query Handlers
    application.add_handler(CallbackQueryHandler(request_approval_callback, pattern=r"^request_approval_\d+$"))
    application.add_handler(CallbackQueryHandler(approval_action_callback, pattern=r"^(approve|reject)_\d+$"))

    # Message Handler for all other messages (text, media, etc.)
    application.add_handler(MessageHandler(filters.ALL & ~filters.COMMAND, handle_message))

    # Error Handler
    application.add_error_handler(error_handler)

    # --- Schedule Jobs ---
    # Inactivity check: Daily at 00:00 UTC
    job_queue.run_daily(check_inactivity, time=datetime.min.time().replace(tzinfo=timezone.utc), name="inactivity_check_job")

    # Reset weekly media count: Every Monday at 00:00 UTC
    # Ensure this runs once a week
    job_queue.run_daily(reset_weekly_media_count, time=datetime.min.time().replace(tzinfo=timezone.utc), days=(0,), name="reset_weekly_media_job") # 0 = Monday

    # Recurring message: Every 3 hours if set
    message_doc = settings_collection.find_one({"_id": "recurring_message"})
    if message_doc and message_doc.get("message_text"):
        job_queue.run_repeating(send_recurring_message, interval=3 * 3600, first=0, name="send_recurring_message_job")

    # Daily Summary: Every day at 23:59 UTC
    job_queue.run_daily(daily_summary, time=datetime.now(timezone.utc).replace(hour=23, minute=59, second=0, microsecond=0), name="daily_summary_job")

    # Weekly Summary: Every Sunday at 23:59 UTC
    job_queue.run_daily(weekly_summary, time=datetime.now(timezone.utc).replace(hour=23, minute=59, second=0, microsecond=0), days=(6,), name="weekly_summary_job") # 6 = Sunday

    # Start Flask health check server in a separate thread
    flask_thread = threading.Thread(target=run_flask_app)
    flask_thread.daemon = True # Allow main program to exit even if thread is running
    flask_thread.start()

    # Start the Bot
    logger.info("Bot started polling...")
    application.run_polling(allowed_updates=Update.ALL_TYPES)


if __name__ == "__main__":
    main()